# info
This notebook finds the gene pairs that are conditionally esential. <br>
Iterate over pairs and state their essentiality likelihood.

In [1]:
import os, pickle, cobra

# 0. user-defined variables

In [2]:
simulation_dir = '/Users/adrian/projects/hpc_results/constrained/double_KO/deployments/'
model_file = '/Users/adrian/projects/mevu/data/model/Recon3DModel_301.mat'
heatmap_info_file = '/Users/adrian/projects/mevu/results/heatmap.doubleKO.pickle'

# 1. simulate original model

In [3]:
%%time
model = cobra.io.load_matlab_model(model_file)

CPU times: user 2min 42s, sys: 486 ms, total: 2min 43s
Wall time: 2min 43s


In [4]:
optimization_results = model.optimize()
original_growth_value = optimization_results.objective_value # 755.0032155506631

In [5]:
print(len(model.genes))
expected_pairs = int(((2248*2248)-2248)/2)
print(expected_pairs)

2248
2525628


# 1. read data

In [6]:
problem_size = len(model.genes)
essentiality = {}
gene_pairs = []

for i in range(problem_size):
    for j in range(problem_size):
        if i < j:
            a = model.genes[i].id
            b = model.genes[j].id
            gene_pair = (a, b)
            
            gene_pairs.append(gene_pair)
            essentiality[gene_pair] = []
                
print('', len(gene_pairs))
print(gene_pairs[:10])

 2525628
[('8639.1', '26.1'), ('8639.1', '314.2'), ('8639.1', '314.1'), ('8639.1', '1591.1'), ('8639.1', '1594.1'), ('8639.1', '10993.1'), ('8639.1', '6818.1'), ('8639.1', '89874.1'), ('8639.1', '160287.1'), ('8639.1', '3939.1')]


In [7]:
simulation_folders = next(os.walk(simulation_dir))[1]
simulation_folders.sort()

## 1.1. iterate over samples

In [8]:
%%time

### for 10 conditions it takes in necio5 [Wall time: 24 min]

for simulation_folder in simulation_folders[:10]:
    
    # 1. read the jar file
    pickle_files = os.listdir(simulation_dir + simulation_folder + '/results/')
    condition_name = pickle_files[0]
    print('working with {}...'.format(condition_name))
    if len(pickle_files) != 1:
        raise ValueError('Found a diffent number of expected files')
    jar = simulation_dir + simulation_folder + '/results/' + condition_name
    f = open(jar,'rb')
    [sampleID, result, double_ko_results] = pickle.load(f)
    f.close()
    
    # 2. add gene essentiality to each pair
    filtered = double_ko_results[double_ko_results['growth'] < original_growth_value/2]
    #print(filtered.shape)
    for pair in filtered['ids']:
        cobra_pair_list = list(pair)
        if len(cobra_pair_list) == 2:
            a = cobra_pair_list[0]; b = cobra_pair_list[1]
            forward = (a, b); reverse = (b, a)

            if forward in gene_pairs:
                essentiality[forward].append(condition_name)
            if reverse in gene_pairs:
                essentiality[reverse].append(condition_name)

working with E-GEOD-30169_GSM752739.cel.pickle...
working with E-GEOD-30169_GSM752740.cel.pickle...
working with E-GEOD-30169_GSM752741.cel.pickle...
working with E-GEOD-30169_GSM752742.cel.pickle...
working with E-GEOD-30169_GSM752743.cel.pickle...
working with E-GEOD-30169_GSM752744.cel.pickle...
working with E-GEOD-30169_GSM752745.cel.pickle...
working with E-GEOD-30169_GSM752746.cel.pickle...
working with E-GEOD-30169_GSM752747.cel.pickle...
working with E-GEOD-30169_GSM752748.cel.pickle...
CPU times: user 24min 38s, sys: 10.2 s, total: 24min 49s
Wall time: 42min 10s


# 2. store data

In [9]:
jar = heatmap_info_file
f = open(jar,'wb')
pickle.dump(essentiality, f)
f.close()